In [20]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score


# 1. Инициализация

In [21]:
np.random.seed(42)
L = 10
K = np.random.uniform(0.2, 0.3, L)
G = 0.25

# 2. Формирование обучающей выборки

In [22]:
windows = []
targets = []
for i in range(7):
    window = K[i:i+3]
    y_i = 1 if K[i+3] >= G else 0
    windows.append(window)
    targets.append(y_i)

X = np.array(windows)
y = np.array(targets)

# 3. Сигмоида

In [23]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# 4. Логарифм функции правдоподобия

In [24]:
def log_likelihood(G, X, y):
    p = sigmoid(X @ G)
    return np.sum(y * np.log(p + 1e-10) + (1 - y) * np.log(1 - p + 1e-10))

# 5. Оптимизация по G (максимизация ℓ(G))

In [25]:
initial_guess = np.zeros(3)
result = minimize(
    fun=lambda G: -log_likelihood(G, X, y),
    x0=initial_guess,
    method='BFGS'
)
G_opt = result.x

# 6. Прогноз вероятности инвестирования для окна K^8 = (K8, K9, K10)

In [26]:
K8 = K[-3:]
p8 = sigmoid(G_opt @ K8)

# 7. Сравнение стратегий

In [27]:
# Смешанная стратегия
y_pred_logistic = (sigmoid(X @ G_opt) >= 0.5).astype(int)

# Чистая стратегия — если последняя ставка в окне >= порога
y_pred_pure = (X[:, -1] >= G).astype(int)

# Оценка точности
acc_logistic = accuracy_score(y, y_pred_logistic)
acc_pure = accuracy_score(y, y_pred_pure)

# 8. Вывод результатов

In [28]:
print("Сгенерированные значения K:", np.round(K, 3))
print("Оптимальный вектор G:", np.round(G_opt, 2))
print("Окно K^8:", np.round(K8, 3))
print("Оцененная вероятность инвестирования p8:", round(p8, 4))
print("Точность (смешанная стратегия):", acc_logistic)
print("Точность (чистая стратегия):", acc_pure)

Сгенерированные значения K: [0.237 0.295 0.273 0.26  0.216 0.216 0.206 0.287 0.26  0.271]
Оптимальный вектор G: [-2588.3    675.54  2054.72]
Окно K^8: [0.287 0.26  0.271]
Оцененная вероятность инвестирования p8: 0.0001
Точность (смешанная стратегия): 1.0
Точность (чистая стратегия): 0.7142857142857143
